In [ ]:
import pandas as pd
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# Logistic Regresison 
import statsmodels
from  statsmodels.discrete.discrete_model import Logit
import sklearn
from sklearn.linear_model import LogisticRegression
# Decision tree
from xgboost import XGBClassifier    
import math
# Deeplearning
import tensorflow as tf

In [ ]:
# This code is used to disable warning texts
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Or ignore specific warnings by category (e.g., FutureWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
cd = os.getcwd()

dictionary = pd.read_excel(os.path.join(cd, 'data_dictionary.xlsx'))
em_sur = pd.read_csv(os.path.join(cd, 'employee_survey_data.csv'))
gen_data = pd.read_csv(os.path.join(cd, 'general_data.csv'))
mnger_data = pd.read_csv(os.path.join(cd, 'manager_survey_data.csv'))
intime = pd.read_csv(os.path.join(cd, 'in_time.csv'))
outime = pd.read_csv(os.path.join(cd, 'out_time.csv'))

In [ ]:
data = gen_data['Attrition'].value_counts()
plt.pie(
    x= data.values,
    labels = data.index,
    autopct='%1.1f%%'
)

> The attrition rate is around 16%

In [ ]:
# Change the label of Attrition column
gen_data['Attrition_bool'] = gen_data['Attrition'].apply(lambda x: 1 if x == "Yes" else 0)

# LOGISTIC REGRESSION

## Preparing In-time and Out-time data

> The in-time df contains the time each employee arrive at the office. <br>
> The out-time df contains the time each employee leave the office. <br>
> Combining these two df together, we can extract the total time each employee spend working at the office. This metrics can indicate the diligence  of each staff. 

In [ ]:
intime = intime.rename(columns={"Unnamed: 0": "EmployeeID"})
outime = outime.rename(columns={"Unnamed: 0": "EmployeeID"})

In [ ]:
# Transpose the df
intime_T = intime.T

# Check for number of null value in each column: intime_T.isnull().sum().values 
# Observation: Number of null values is quite small, its ok to fill them

# For each employee, fillna value according to the mode of their in-time
for employee in intime_T:
    value_to_fill = intime_T[employee].value_counts().sort_values(ascending=False).index[1]
    intime_T[employee] = intime_T[employee].fillna(value_to_fill)

In [ ]:
for employee in intime_T:
    datetime_data = pd.to_datetime(intime_T[employee]) # convert to datetime dtype
    hour_data = datetime_data.dt.strftime('%H').apply(lambda x: int(x)) # get current hour, convert to integer
    min_data = datetime_data.dt.strftime('%M').apply(lambda x: int(x)) # get current minute, convert to integer
    data_to_return = hour_data * 60 + min_data # get current minute in day
    intime_T[employee] = data_to_return # updatee the column 

In [ ]:
# Transpose the df
outime_T = outime.T

# Check for number of null value in each column: intime_T.isnull().sum() 
# Observation: Number of null values is quite small, its ok to fill them

# For each employee, fillna value according to the mode of their in-time
for employee in outime_T:
    value_to_fill = outime_T[employee].value_counts().sort_values(ascending=False).index[1]
    outime_T[employee] = outime_T[employee].fillna(value_to_fill)


In [ ]:
for employee in outime_T:
    datetime_data = pd.to_datetime(outime_T[employee]) # convert to datetime dtype
    hour_data = datetime_data.dt.strftime('%H').apply(lambda x: int(x)) # get current hour, convert to integer
    min_data = datetime_data.dt.strftime('%M').apply(lambda x: int(x)) # get current minute, convert to integer
    data_to_return = hour_data * 60 + min_data # get current minute in day
    outime_T[employee] = data_to_return # updatee the column 

In [ ]:
office_time_spent = outime_T - intime_T # get the time spent
office_time_spent = office_time_spent.iloc[1:,:] # delete the employeeID row
agg_time_spent = pd.DataFrame(office_time_spent.T.mean(axis=1),columns=['Time_spent']) # get the mean of time spent
agg_time_spent

## Concatenate all variables in one data frame

In [ ]:
# Ratio dataframe
gen_data_new = pd.concat([gen_data,agg_time_spent],axis=1)

In [ ]:
ratio_data = gen_data_new[['Time_spent', 'Age','DistanceFromHome','MonthlyIncome','NumCompaniesWorked',
                           'PercentSalaryHike','TotalWorkingYears','TrainingTimesLastYear',
                           'YearsAtCompany','YearsSinceLastPromotion','YearsWithCurrManager',
                           'Attrition_bool']]

In [ ]:
merged_df = em_sur.merge(gen_data, on='EmployeeID').merge(mnger_data, on='EmployeeID')
likert_data = merged_df[['JobInvolvement','JobLevel','JobSatisfaction','EnvironmentSatisfaction',
                        'WorkLifeBalance','PerformanceRating','Attrition_bool']]

## Define function performing Logistic Regression and outputing coeff table

In [ ]:
def logit_ratio(df):
    dict = {}
    for var in df.drop(columns = 'Attrition_bool'):
        data = df[[var,'Attrition_bool']].dropna()
        # get x, y
        x = data[var].values
        y = data['Attrition_bool'].values
        # standardize 
        x = (x - x.mean()) / (x.std())
        x = np.expand_dims(x,axis=-1)
        # fit the model
        model = LogisticRegression().fit(x,y)
        coeff = model.coef_
        coeff = np.squeeze(coeff,axis=1)
        dict[var] = coeff
        # gen result df
        res_df = pd.DataFrame(dict).T.rename(columns={0: "Coefficient"})
        res_df['Absolute_val'] = res_df.apply(lambda x: np.abs(x))
        res_df = res_df.sort_values('Absolute_val',ascending=False)
    return res_df

##  Execute Logistic Regession

In [ ]:
display(
    "RATIO DATA COEFF", logit_ratio(ratio_data),
    "LIKERT - INTERVAL DATA COEFF", logit_ratio(likert_data)
)


## Visulize and analyze

In [ ]:
vis_df = pd.concat([logit_ratio(ratio_data),logit_ratio(likert_data)]).sort_values('Absolute_val',ascending=True) # concat to 1 df and sort 

In [ ]:
sns.set_theme(style='white')
fig, ax = plt.subplots(figsize =(7,7))
# color map edit
cm = ['#FF8080'] * len(vis_df.index)
cm[3] = cm[4] = cm[7] = cm[14] = '#CDFAD5'

ax.barh(
    y = vis_df.index,
    width = vis_df['Absolute_val'],
    color = cm
)

# annotate
for var_name, var_val in vis_df.iterrows():
    ax.annotate(
        str(var_val['Coefficient'])[:5],
        xy = (var_val['Absolute_val'] + 0.01, var_name),
        va='center',ha='left', fontweight = 'medium'
    )    

# title and labels
ax.set_yticklabels(vis_df.index ,fontweight = 'medium')
ax.set_title(
    "Factors' Coefficience",
    fontsize = 15,
    fontweight = 'bold'
)

sns.despine()


## Conclusion and next step

> Observations:
> <ol>
> - Tenure has the biggest effect on the attrition rate. Variables that indicate employees' tenure include: 'Total Working Years', 'Years with current manager', 'Years at company', even > 'Age'. This result indicates that the longer employees stick to this firm, the less likely they are fired or leaving the company on themselves. <br>
> - Oddly, the longer time employees spend on office a day, the more likely they leave the position. Coefficience for 'Time spent' variable is 0.498, among the most predictive factors. <br>
> - Satisfaction is very important. Factors such as 'Employee satisfaction', 'Environment satisfaction', 'Worklife balance' have notable effect on Attrition rate.<br>
> - On the opposite, physical value like promotion, monthly income do not influence much on Attrition rate.  <br>
> </ol>
> Next step:
> We will continue evaluating these relationships using feature importance extracted from decision tree.

# DECISION TREE

## Evaluate feature importance (Excluding Categorical data)

In [ ]:
# Prepare input
ratio_data['EmployeeID'] = range(1,len(ratio_data)+1) # create key to merge
likert_data['EmployeeID'] = range(1,len(likert_data)+1) # create key to merge

data = ratio_data.merge(likert_data, on = 'EmployeeID')
data = data.drop(columns=['EmployeeID', "Attrition_bool_x"]).rename(columns={'Attrition_bool_y': "Attrition_bool"}) # merge / drop / rename

In [ ]:
# get data
x = data.drop(columns='Attrition_bool')
y = data['Attrition_bool']
# split to train and test
split_n = math.floor(len(data) * 0.8)
xtrain = x.iloc[:split_n, :]
ytrain = np.expand_dims(y[:split_n].values,axis=-1)
xtest = x.iloc[split_n:, :]
ytest = np.expand_dims(y[split_n:].values,axis=-1)


# build model
model = XGBClassifier()
# train model
model.fit(
    xtrain,
    ytrain,
    eval_set=[(xtest, ytest)],
    verbose = 0
)

In [ ]:
feature_importance = model.feature_importances_
feature_names = data.drop(columns='Attrition_bool').columns

In [ ]:
# Get feature importance dataframe - All variables
feature_importance = model.feature_importances_
feature_importance_list = [[fi] for fi in list(feature_importance)]
feature_names = data.drop(columns='Attrition_bool').columns
feature_importance_dict = dict(zip(feature_names, feature_importance_list))
feature_importance_df = pd.DataFrame(feature_importance_dict).T.reset_index().rename(columns={0: "Importance", "index": "Variable"}).sort_values('Importance')
# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 7))
ax.barh(
    feature_importance_df['Variable'], 
    feature_importance_df['Importance'],
    color = "#7A9D54"
    )
##  Annotating
for index, row in feature_importance_df.iterrows():
    ax.annotate(
        str(row['Importance'])[:5],
        xy = (row['Importance'],row['Variable'])
    )
ax.set_xticklabels([])
ax.set_xlabel("Importance scores")
ax.set_ylabel('Variable')
ax.set_title(
    """
    FEATURE IMPORTANCE
    Excluding Categorial Variables
    """,
    fontsize = 15, 
    fontweight = "bold"
)
sns.despine()


> Observations:
> <ol>
> - Again, Tenure has the biggest effect on the attrition rate, as indicated by the coefficience of factor 'Total Working Years'. <br>
> - Unusually, extracted feature importance demonstrates that 'Years since last promotion' and 'Job involvement' are 2 influential factors. <br>
> - Overall, result from feature improtance obtained through decision tree has similar trend with that of Logistic regresison. <br>
> </ol>
> Next step: We will add in the decision tree model new Categorical features to see their importance

## Evaluate feature importance (Including Categorical data)

In [ ]:
# Prepare input
categorical_data = gen_data[['BusinessTravel','Education','Department','EducationField','Gender','JobRole','MaritalStatus','Over18']]

ratio_data['EmployeeID'] = range(1,len(ratio_data)+1) # create key to merge
likert_data['EmployeeID'] = range(1,len(likert_data)+1) # create key to merge
categorical_data['EmployeeID'] = range(1,len(ratio_data)+1) # create key to merge

data = ratio_data.merge(likert_data, on = 'EmployeeID').merge(categorical_data, on = 'EmployeeID')
data = data.drop(columns=['EmployeeID', "Attrition_bool_x"]).rename(columns={'Attrition_bool_y': "Attrition_bool"}) # merge / drop / rename
data = pd.get_dummies(data)

In [ ]:
model = XGBClassifier()
# get data
x = data.drop(columns='Attrition_bool')
y = data['Attrition_bool']
# split to train and test
split_n = math.floor(len(data) * 0.8)
xtrain = x.iloc[:split_n, :]
ytrain = np.expand_dims(y[:split_n].values,axis=-1)
xtest = x.iloc[split_n:, :]
ytest = np.expand_dims(y[split_n:].values,axis=-1)
# trainn model
model.fit(
    xtrain,
    ytrain,
    eval_set=[(xtest, ytest)],
    verbose=0
)

In [ ]:
feature_importance = model.feature_importances_
feature_names = data.drop(columns='Attrition_bool').columns

In [ ]:
# Get feature importance dataframe - All variables
feature_importance = model.feature_importances_
feature_importance_list = [[fi] for fi in list(feature_importance)]
feature_names = data.drop(columns='Attrition_bool').columns
feature_importance_dict = dict(zip(feature_names, feature_importance_list))
feature_importance_df = pd.DataFrame(feature_importance_dict).T.reset_index().rename(columns={0: "Importance", "index": "Variable"}).sort_values('Importance')
# Plot feature importance
fig, ax2 = plt.subplots(figsize=(10, 13))
ax2.barh(
    feature_importance_df['Variable'], 
    feature_importance_df['Importance'],
    color = '#7A9D54'
    )
##  Annotating
for index, row in feature_importance_df.iterrows():
    ax2.annotate(
        str(row['Importance'])[:5],
        xy = (row['Importance'] + 0.001,row['Variable']),
        va = 'center', ha='left'
    )
ax2.set_xticklabels([])
ax2.set_xlabel("Importance scores")
ax2.set_ylabel('Variable')
ax2.set_title(
    """
    FEATURE IMPORTANCE 
    Including Categorical Variable 
    """, 
    fontsize = 15, 
    fontweight = "bold"
)
sns.despine()



> Lets explore the 'EducationField_Marketing' and 'MaritalStatus_Single' features 

In [ ]:
# This fucntion help calculate and visulize relationship between categorical variables 
from sklearn.metrics import confusion_matrix
def explore_cate_data(cat_var):
    cm = confusion_matrix(data[cat_var], data['Attrition_bool'])
    attrition_rate = cm[1,1] / cm[1,0]
    
    sns.heatmap(cm, cmap="viridis")   
    ax = plt.gca()
    ax.set_xlabel('Attrition_bool')
    ax.set_ylabel(cat_var)

    print(f'When {cat_var} is true, attrition rate is {attrition_rate}')
    print(cm)
    plt.show()


In [ ]:
explore_cate_data('EducationField_Marketing')

> There is no obvious relationship between employees coming from Marketing field and their attrition rate

In [ ]:
explore_cate_data('MaritalStatus_Single')

> When the marital status of employees is single, there is 34% chance they will leave the company on the next year. <br>
> This number is nearly double the average attrition rate, which is 15%. <br>
> 
> <strong> This result indicates that a un-married employee has a higher chance of quiting or being fired. <br> </strong>

# Recommendation

**From the Company's Perspective:**

- **Reward System for Tenure:** Data demonstrates the significant influence of tenure on attrition rates. When an employee's total working years with the company increase by 1, the likelihood of them leaving the organization decreases by a substantial 11.69%.
- **Emphasizing Employee Satisfaction:** The findings also underscore the crucial role of employee satisfaction in relation to job contentment, work environment, and work-life balance. To effectively mitigate attrition, the company should prioritize initiatives aimed at enhancing employee satisfaction.

**From the Employee's Perspective:**

- **Consider Avoiding Candidates with a High Company-Change Rate:** It's important to take into account an applicant's history of frequent company changes, as this factor has a positive correlation with attrition rates.
- **Preference for Married Candidates:** Of noteworthy concern is the attrition rate among single employees, with approximately 1/3 of them having left the company in the previous year. This figure is double the average attrition rate, suggesting a preference for candidates in marital relationships may be beneficial.